<a href="https://colab.research.google.com/github/drshahizan/Python-big-data/blob/main/assignment/ass6/hpdp/vtm/Assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6

| NAME | MATRIC NO. |
| --- | --- |
| MOHAMAD AZRI HADIF BIN MOHAMMAD RIZAL | A21EC0054 |
| MUHAMMAD IQMAL BIN SIS | A21EC0080|


## Import Dataset

In [1]:
from google.colab import files

files.upload()
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d arevel/chess-games

Saving kaggle.json to kaggle.json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
100% 1.45G/1.45G [00:15<00:00, 178MB/s]
100% 1.45G/1.45G [00:15<00:00, 103MB/s]


### Context
Chess is one of the more famous board games of the world and one of the most studied. With it's more than 4 GB of information this can be a good opportunity to work with a huge dataset and practice on how to handle it.

### Content
This dataset contains 6.25 Million chess games played on lichess.org during July of 2016.
Some of the games have Stockfish analysis evaluations like* [%eval 2.35] (235 centipawn advantage)* always from White's point of view. These are evaluations of the movement made by a player.

- Event: Game type.
- White: White's ID.
- Black: Black's ID.
- Result: Game Result (1-0 White wins) (0-1 Black wins)
- UTCDate: UTC Date.
- UTCTime: UTC Time.
- WhiteElo: White's ELO.
- BlackElo: Black's ELO.
- WhiteRatingDiff: White's rating points difference after the game.
- BlackRatingDiff: Blacks's rating points difference after the game.
- ECO: Opening in ECO encoding.
- Opening: Opening name.
- TimeControl: Time of the game for each player in seconds. The number after the increment is the number of seconds before the player's clock starts ticking in each turn.
- Termination: Reason of the game's end.
- AN: Movements in Movetext format.

Acknowledgements
The original data has been published by [lichess.org](https://lichess.org/).

In [2]:
from zipfile import ZipFile

In [3]:
filepath = "/content/chess-games.zip"

with ZipFile(filepath) as zip:
  zip.printdir()
  zip.extractall()

!rm /content/chess-games.zip

File Name                                             Modified             Size
chess_games.csv                                2021-02-15 16:00:32   4378895157


In [4]:
# import necessary library
import pandas as pd
import numpy as np
import time
import os
import psutil

In [5]:
# Read data

df = pd.read_csv("/content/chess_games.csv")
df.head()

,Event,White,Black,Result,UTCDate,UTCTime,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,AN
0,Classical,eisaaaa,HAMID449,1-0,2016.06.30,22:00:01,1901,1896,11.0,-11.0,D10,Slav Defense,300+5,Time forfeit,1. d4 d5 2. c4 c6 3. e3 a6 4. Nf3 e5 5. cxd5 e...
1,Blitz,go4jas,Sergei1973,0-1,2016.06.30,22:00:01,1641,1627,-11.0,12.0,C20,King's Pawn Opening: 2.b3,300+0,Normal,1. e4 e5 2. b3 Nf6 3. Bb2 Nc6 4. Nf3 d6 5. d3 ...
2,Blitz tournament,Evangelistaizac,kafune,1-0,2016.06.30,22:00:02,1647,1688,13.0,-13.0,B01,Scandinavian Defense: Mieses-Kotroc Variation,180+0,Time forfeit,1. e4 d5 2. exd5 Qxd5 3. Nf3 Bg4 4. Be2 Nf6 5....
3,Correspondence,Jvayne,Wsjvayne,1-0,2016.06.30,22:00:02,1706,1317,27.0,-25.0,A00,Van't Kruijs Opening,-,Normal,1. e3 Nf6 2. Bc4 d6 3. e4 e6 4. Nf3 Nxe4 5. Nd...
4,Blitz tournament,kyoday,BrettDale,0-1,2016.06.30,22:00:02,1945,1900,-14.0,13.0,B90,"Sicilian Defense: Najdorf, Lipnitsky Attack",180+0,Time forfeit,1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. N...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6256184 entries, 0 to 6256183
Data columns (total 15 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Event            object 
 1   White            object 
 2   Black            object 
 3   Result           object 
 4   UTCDate          object 
 5   UTCTime          object 
 6   WhiteElo         int64  
 7   BlackElo         int64  
 8   WhiteRatingDiff  float64
 9   BlackRatingDiff  float64
 10  ECO              object 
 11  Opening          object 
 12  TimeControl      object 
 13  Termination      object 
 14  AN               object 
dtypes: float64(2), int64(2), object(11)
memory usage: 716.0+ MB


## Load Less Data:
Strategically load only the essential portions of the dataset to optimize memory usage.

In [7]:
# Measure the start time
start_time = time.time()

# Load only the specified columns
selected_columns = ['Event', 'Result', 'WhiteElo', 'BlackElo', 'WhiteRatingDiff', 'BlackRatingDiff', 'Opening', 'Termination']
df_selected = pd.read_csv('chess_games.csv', usecols = selected_columns)

# Measure the end time
end_time = time.time()


To assess the status of someone's success in chess games, we have narrowed our focus to specific columns that are highly relevant to our objective. This approach not only streamlines the analysis but also minimizes the time required for the machine to perform the assessment as it optimize the usage of the machine.

1. Selecting the column
        Selected Column: ['event', 'result', 'whiteElo', 'BlackElo', 'whiteratingdiff', 'blackratingdiff', 'opening', 'termination']


2. Reading the CSV File:

* pd.read_csv('chess_games.csv', usecols=selected_columns): This line uses the read_csv function from pandas to read data from a CSV file.

* The usecols parameter is used to specify which columns should be loaded from the CSV file. In this case, only the columns listed in selected_columns will be loaded into the DataFrame_Selected (df_selected).

In [8]:
df_selected.head()

,Event,Result,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,Opening,Termination
0,Classical,1-0,1901,1896,11.0,-11.0,Slav Defense,Time forfeit
1,Blitz,0-1,1641,1627,-11.0,12.0,King's Pawn Opening: 2.b3,Normal
2,Blitz tournament,1-0,1647,1688,13.0,-13.0,Scandinavian Defense: Mieses-Kotroc Variation,Time forfeit
3,Correspondence,1-0,1706,1317,27.0,-25.0,Van't Kruijs Opening,Normal
4,Blitz tournament,0-1,1945,1900,-14.0,13.0,"Sicilian Defense: Najdorf, Lipnitsky Attack",Time forfeit


In [9]:
df_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6256184 entries, 0 to 6256183
Data columns (total 8 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Event            object 
 1   Result           object 
 2   WhiteElo         int64  
 3   BlackElo         int64  
 4   WhiteRatingDiff  float64
 5   BlackRatingDiff  float64
 6   Opening          object 
 7   Termination      object 
dtypes: float64(2), int64(2), object(4)
memory usage: 381.8+ MB


Memory Usage of df_selected

In [10]:
# Calculate memory usage
memory_usage = df_selected.memory_usage().sum() / (1024 ** 2)  # Convert bytes to megabytes

Computation Time of df_selected

In [11]:
# Calculate computation time
computation_time = end_time - start_time

File Size of df_selected

In [12]:
# Calculate file size
file_size = os.path.getsize('chess_games.csv') / (1024 ** 2)  # Convert bytes to megabytes

Result:

In [13]:
# Print the results
print(f"Memory Usage: {memory_usage:.2f} MB")
print(f"Computation Time: {computation_time:.2f} seconds")
print(f"File Size: {file_size:.2f} MB")

Memory Usage: 381.85 MB
Computation Time: 58.73 seconds
File Size: 4176.04 MB


## Use Chunking:
Process the data in smaller pieces to avoid memory issues.

We decided to use 750,000 as our chunk size. The chunking process is a process where it separated the file size and store it into different parts. It divide them for a more easier process for the computer or machine to operate the analysis. The chunking used the looping process as the it only divide the same thing over and over again:- Using the for loop

      for chunk in pd.read_csv('chess_games.csv', chunksize=chunk_size):

Then, we calculate and measure the File size, Memory usage, and computation time for each chunk process.

In [14]:
# Define chunk size

chunk_size = 750000

# Measure the start time
start_time = time.time()

# Initialize a variable to store the total number of chunks
total_chunks = 0

# Initialize variables for file size, computation time, and memory usage
total_file_size = 0
total_computation_time = 0
total_memory_usage = 0

# Process data in chunks
for chunk in pd.read_csv('chess_games.csv', chunksize=chunk_size):
    # Count the number of chunks
    total_chunks += 1

    # Display information about the chunk
    print(f"Processing Chunk {total_chunks}:")
    print(chunk.head())  # Display the first few rows of the chunk

    # Calculate memory usage
    memory_usage = chunk.memory_usage().sum() / (1024 ** 2)  # Convert bytes to megabytes
    print(f"Memory Usage: {memory_usage:.2f} MB")

    # Measure the file size
    file_size = os.path.getsize('chess_games.csv') / (1024 ** 2)  # Convert bytes to megabytes
    print(f"File Size: {file_size:.2f} MB")

    # Measure the computation time for each chunk
    computation_time = time.time() - start_time
    print(f"Computation Time: {computation_time:.2f} seconds\n")

    # Accumulate total values for file size, computation time, and memory usage
    total_file_size += file_size
    total_computation_time += computation_time
    total_memory_usage += memory_usage

# Print overall statistics
print(f"Total Chunks: {total_chunks}")
print(f"Total File Size: {total_file_size:.2f} MB")
print(f"Total Computation Time: {total_computation_time:.2f} seconds")
print(f"Total Memory Usage: {total_memory_usage:.2f} MB")

Processing Chunk 1:
                Event            White       Black Result     UTCDate  \
0          Classical           eisaaaa    HAMID449    1-0  2016.06.30   
1              Blitz            go4jas  Sergei1973    0-1  2016.06.30   
2   Blitz tournament   Evangelistaizac      kafune    1-0  2016.06.30   
3     Correspondence            Jvayne    Wsjvayne    1-0  2016.06.30   
4   Blitz tournament            kyoday   BrettDale    0-1  2016.06.30   

    UTCTime  WhiteElo  BlackElo  WhiteRatingDiff  BlackRatingDiff  ECO  \
0  22:00:01      1901      1896             11.0            -11.0  D10   
1  22:00:01      1641      1627            -11.0             12.0  C20   
2  22:00:02      1647      1688             13.0            -13.0  B01   
3  22:00:02      1706      1317             27.0            -25.0  A00   
4  22:00:02      1945      1900            -14.0             13.0  B90   

                                         Opening TimeControl   Termination  \
0                 

## Optimize Data Types:
Fine-tune data types to maximize efficiency and minimize memory consumption.

Optimizing the data types is crucially in making sure the machine can run faster thus cutting the cost and also time for analysis.

In our case, we 'downsize' the integer from **int64** to **int16** depending on the data itself. Next we also change the **object** data type to **category**. By doing this, we make sure that by reducing memory usage when dealing with large datasets, it allows for more efficient storage and faster processing.

    # Convert numeric columns to more memory-efficient types
      numeric_cols = ['WhiteElo', 'BlackElo', 'WhiteRatingDiff', 'BlackRatingDiff']
      df_optimized[numeric_cols] = df_optimized[numeric_cols].apply(pd.to_numeric, downcast='integer', errors='coerce')

    # Convert object columns to category if applicable
      object_cols = ['Event', 'Result', 'Opening', 'Termination']
    df_optimized[object_cols] = df_optimized[object_cols].astype('category')

In [15]:
df_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6256184 entries, 0 to 6256183
Data columns (total 8 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Event            object 
 1   Result           object 
 2   WhiteElo         int64  
 3   BlackElo         int64  
 4   WhiteRatingDiff  float64
 5   BlackRatingDiff  float64
 6   Opening          object 
 7   Termination      object 
dtypes: float64(2), int64(2), object(4)
memory usage: 381.8+ MB


In [16]:
# Measure the start time
start_time = time.time()

# Display the original memory usage
original_memory_usage = df_selected.memory_usage(deep=True).sum() / (1024 ** 2)  # Convert bytes to megabytes
print(f"Original Memory Usage: {original_memory_usage:.2f} MB")

# Optimize data types
df_optimized = df_selected.copy()

# Convert numeric columns to more memory-efficient types
numeric_cols = ['WhiteElo', 'BlackElo', 'WhiteRatingDiff', 'BlackRatingDiff']
df_optimized[numeric_cols] = df_optimized[numeric_cols].apply(pd.to_numeric, downcast='integer', errors='coerce')

# Convert object columns to category if applicable
object_cols = ['Event', 'Result', 'Opening', 'Termination']
df_optimized[object_cols] = df_optimized[object_cols].astype('category')


# Measure the end time
end_time = time.time()

# Display the optimized memory usage
optimized_memory_usage = df_optimized.memory_usage(deep=True).sum() / (1024 ** 2)  # Convert bytes to megabytes
print(f"Optimized Memory Usage: {optimized_memory_usage:.2f} MB")

# Compare the memory usage reduction
memory_reduction = (original_memory_usage - optimized_memory_usage) / original_memory_usage * 100
print(f"Memory Reduction: {memory_reduction:.2f}%")

Original Memory Usage: 1864.15 MB
Optimized Memory Usage: 149.51 MB
Memory Reduction: 91.98%


In [17]:
df_optimized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6256184 entries, 0 to 6256183
Data columns (total 8 columns):
 #   Column           Dtype   
---  ------           -----   
 0   Event            category
 1   Result           category
 2   WhiteElo         int16   
 3   BlackElo         int16   
 4   WhiteRatingDiff  float64 
 5   BlackRatingDiff  float64 
 6   Opening          category
 7   Termination      category
dtypes: category(4), float64(2), int16(2)
memory usage: 149.2 MB


Computation Time for Optimize Data Type Operations

In [18]:
# Calculate computation time
computation_time = end_time - start_time

Memory Usage for Optimize Data Type Operation

In [19]:
# Calculate memory usage for the optimized DataFrame
optimized_memory_size = df_optimized.memory_usage(deep=True).sum()

# Convert bytes to megabytes
optimized_memory_size_mb = optimized_memory_size / (1024 ** 2)

Result

In [20]:
print(f"Computation Time: {computation_time:.2f} seconds")
print(f"Optimized Memory Size: {optimized_memory_size_mb:.2f} MB")

Computation Time: 7.78 seconds
Optimized Memory Size: 149.51 MB


## Sampling:
Implement sampling methodologies to extract meaningful insights from a subset of the dataset.

Sampling is a method where we take a subset from an original dataset (df_selected). There are 2 type of sampling that we done which are **Random Sampling** and **Stratified Sampling**.

**Random Sampling** is a method where it randomizely take 10% of the whole dataset. Based on the code given:

    random_sampled_data = df_selected.sample(frac=0.1, random_state=42)

**Stratified Sampling** is a method where it also take 10% of the whole dataset. But, it will keep the unique values of the category column data type. This is advantageous to potray the representation for each category of the dataset.

    stratified_sampled_data = df_selected.groupby(category_column, group_keys=False).apply(lambda x: x.sample(frac=0.1, random_state=42))

In [21]:
category_column = ['Event', 'Result', 'Opening', 'Termination']

start_time = time.time()

# Random sampling (10% of the DataFrame)
random_sampled_data = df_selected.sample(frac=0.1, random_state=42)  # Adjust random_state if reproducibility is needed

# Measure the end time for random sampling
random_sampling_time = time.time() - start_time

# Stratified sampling (10% within each category of 'category_column')
start_time = time.time()
stratified_sampled_data = df_selected.groupby(category_column, group_keys=False).apply(lambda x: x.sample(frac=0.1, random_state=42))

# Measure the end time for stratified sampling
stratified_sampling_time = time.time() - start_time



Memory usage for random sampling


In [22]:
# Calculate memory usage for random sampling
random_memory_usage = random_sampled_data.memory_usage().sum() / (1024 ** 2)  # Convert bytes to megabytes

Memory usage for stratified sampling

In [23]:
# Calculate memory usage for stratified sampling
stratified_memory_usage = stratified_sampled_data.memory_usage().sum() / (1024 ** 2)  # Convert bytes to megabytes

File size for Random sampling

In [24]:
# Random Sample File Size
csv_representation = random_sampled_data.to_csv(index=False)
csv_size = len(csv_representation.encode('utf-8')) / (1024 ** 2)  # Convert bytes to megabytes

File size for Stratified sampling

In [25]:
# Stratified Sampling File Size
csv_representation2 = stratified_sampled_data.to_csv(index=False)
csv_size2 = len(csv_representation2.encode('utf-8')) / (1024 ** 2)  # Convert bytes to megabytes

Result

In [26]:
# Print the results
print(f"\nRandom Sampling:")
print(f"Memory Usage: {random_memory_usage:.2f} MB")
print(f"Computation Time: {random_sampling_time:.2f} seconds")

print(f"\nStratified Sampling:")
print(f"Memory Usage: {stratified_memory_usage:.2f} MB")
print(f"Computation Time: {stratified_sampling_time:.2f} seconds")

print(f"\nRandom Sampling File Size: {csv_size:.2f} MB")
print(f"\nStratified Sampling File Size: {csv_size:.2f} MB")


Random Sampling:
Memory Usage: 42.96 MB
Computation Time: 0.80 seconds

Stratified Sampling:
Memory Usage: 42.60 MB
Computation Time: 63.07 seconds

Random Sampling File Size: 45.80 MB

Stratified Sampling File Size: 45.80 MB


## Paralellize Using Dask

Install Dask

In [27]:
# installing dask
!pip install "dask[complete]"    # Install everything

Import necessary library from Dask, in this case dask.dataframe

In [28]:
import dask.dataframe as dd

Specify file_path

In [29]:
file_path = 'chess_games.csv'

Calculate file size in MB

In [30]:
file_size_dask = os.path.getsize(file_path) / (1024 * 1024)
print(f"File size: {file_size_dask} MB")

File size: 4176.039845466614 MB


Calculate time taken to read file using dd.read_csv

In [31]:
start_time = time.time()

dask_df = dd.read_csv(file_path)

end_time = time.time()
time_taken = end_time - start_time
print(f"Time taken to read file using Dask: {time_taken}")

Time taken to read file using Dask: 0.021546363830566406


Calculate memory usage and time taken when using dask to compute calculation in this case, mean of Black's ELO

In [32]:
memory_before = psutil.Process().memory_info().rss / 1024 / 1024
start_time = time.time()

mean_black_elo = dask_df['BlackElo'].mean().compute()

end_time = time.time()
memory_after = psutil.Process().memory_info().rss / 1024 / 1024

In [33]:
memory_used = memory_after - memory_before

print(f"Memory used to calculate mean using Dask: {memory_used} MB")
print(f"Time taken to calculate mean using Dask: {time_taken} seconds")

Memory used to calculate mean using Dask: 138.28515625 MB
Time taken to calculate mean using Dask: 0.021546363830566406 seconds


## Comparative Analysis

Strategy
- Load Less Data

  Memory Usage: 381.85 MB  
  Computation Time: 61.41 seconds  
  File Size: 4176.04 MB

- Chunking

  Total File Size is 37584.36 MB  
  Total Computation Time: 528.38 seconds  
  Total Memory Usage: 715.96 MB  

- Data Type Optimization

  Original Memory Usage: 1864.15 MB  
  Optimized Memory Usage: 149.51 MB  
  Memory Reduction: 91.98%  
  Computation Time: 9.36 seconds  

- Sampling

  Random Sampling:

  - Memory Usage: 42.96 MB  
  - Computation Time: 0.84 seconds  

  Stratified Sampling:

  - Memory Usage: 42.60 MB  
  - Computation Time: 72.16 seconds  

  Random Sampling File Size: 45.80 MB  

  Stratified Sampling File Size: 45.80 MB  

- Parallelize with Dask

  File size: 4176.039845466614 MB  
  Time taken to read file using Dask: 0.02105093002319336  
  Memory used to calculate mean using Dask: 9.3203125 MB  
  Time taken to calculate mean using Dask: 0.02105093002319336 seconds  

## Conclusion

1. Data Loading:  
Objective: The strategy involves loading only the necessary data, thereby reducing both memory usage and computation time.  

  Advantages:
  - Minimizes the amount of data loaded into memory.
  - Reduces the time spent on loading unnecessary data.

2. Chunking:  
  Objective: Divide large files into smaller chunks to facilitate processing in more manageable pieces.

  Advantages:
  - Enables processing of large datasets that may not fit into memory.
  - Supports parallel processing on smaller chunks.

3. Data Type Optimization:  
  Objective: Optimize the data types of variables to use less memory while maintaining data integrity.  
  Advantages:
  - Reduces memory usage significantly, especially for large datasets.
  - Improves computation time by minimizing the amount of data transferred.

4. Sampling:  
  Objective: Select a subset of data points from the entire dataset for analysis.

  Advantages:
  - Reduces memory usage and computation time by working with a smaller sample.
  - Random sampling is quicker and more memory-efficient than stratified sampling.

5. Parallelization with Dask:  
  Objective: Leverage Dask for parallel and distributed computing to process data more efficiently.

  Advantages:
  - Enables parallel processing, reducing overall computation time.
  - Distributes computation across multiple cores or machines, addressing memory constraints.


Each strategy addresses specific challenges related to memory usage and computation time. The choice of strategy depends on the nature of the data, the analytical tasks at hand, and the available computing resources.